## UFO Sightings Evaluation and Optimization Lab

The goal of this notebook is to find out if our optimized model hyperparmeters out performs the training of our baseline Linear Learner model. We can also compare things like accurary and see if they differ.

What we plan on accompishling is the following:
1. [Create and train our "optimized" model (Linear Learner)](#1.-Create-and-train-our-%22optimized%22-model-(Linear-Learner))
1. Compare the results!

First let's go ahead and import all the needed libraries.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime


import boto3
from sagemaker import get_execution_role
import sagemaker

In [2]:
role = get_execution_role()
bucket='sagemaker-js/DemoLabUFO'

---

### 1. Create and train our "optimized" model (Linear Learner)

Let's evaluate the Linear Learner algorithm with the new optimized hyperparameters. Let's go ahead and get the data that we already stored into S3 as recordIO protobuf data.

Let's get the recordIO file for the training data that is in S3

In [3]:
train_file = 'ufo_sightings_train_recordIO_protobuf.data'
training_recordIO_protobuf_location = 's3://{}/algorithms_lab/linearlearner_train/{}'.format(bucket, train_file)
print('The Pipe mode recordIO protobuf training data: {}'.format(training_recordIO_protobuf_location))

The Pipe mode recordIO protobuf training data: s3://sagemaker-js/DemoLabUFO/algorithms_lab/linearlearner_train/ufo_sightings_train_recordIO_protobuf.data


Let's get the recordIO file for the validation data that is in S3

In [4]:
validation_file = 'ufo_sightings_validatioin_recordIO_protobuf.data'
validate_recordIO_protobuf_location = 's3://{}/algorithms_lab/linearlearner_validation/{}'.format(bucket, validation_file)
print('The Pipe mode recordIO protobuf validation data: {}'.format(validate_recordIO_protobuf_location))

The Pipe mode recordIO protobuf validation data: s3://sagemaker-js/DemoLabUFO/algorithms_lab/linearlearner_validation/ufo_sightings_validatioin_recordIO_protobuf.data


---

Alright we are good to go for the Linear Learner algorithm. Let's get everything we need from the ECR repository to call the Linear Learner algorithm.

In [5]:
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker

container = get_image_uri(boto3.Session().region_name, 'linear-learner', "1")

Let's create a job and use the optimzed hyperparamters.

In [6]:
# Create a training job name
job_name = 'ufo-linear-learner-job-optimized-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))

# Here is where the model-artifact will be stored
output_location = 's3://{}/optimization_evaluation_lab/linearlearner_optimized_output'.format(bucket)

Here is the job name ufo-linear-learner-job-optimized-20190725131026


Next we can start building out our model by using the SageMaker Python SDK and passing in everything that is required to create a Linear Learner training job.

Here are the [linear learner hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) that we can use within our training job.

After we run this job we can view the results.

In [9]:
%%time
sess = sagemaker.Session()

# Setup the LinearLeaner algorithm from the ECR container
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess,
                                       input_mode='Pipe')
# Setup the hyperparameters
linear.set_hyperparameters( feature_dim=22, 
                            predictor_type='multiclass_classifier',
                            num_classes=3,
                            early_stopping_patience=3,
                            early_stopping_tolerance=0.001,
                            epochs=15,
                            l1=0.1333817642465768,
                            loss='auto',
                            mini_batch_size=692,
                            normalize_data='true',
                            normalize_label='auto',
                            num_models='auto',
                            optimizer='auto',
                            unbias_data='auto',
                            unbias_label='auto',
                            use_bias='true',
                            wd=0.008898628555711788
                          )


# Launch a training job. This method calls the CreateTrainingJob API call
data_channels = {
    'train': training_recordIO_protobuf_location,
    'validation': validate_recordIO_protobuf_location
}
linear.fit(data_channels, job_name=job_name)

2019-06-26 13:36:13 Starting - Starting the training job...
2019-06-26 13:36:14 Starting - Launching requested ML instances......
2019-06-26 13:37:46 Starting - Preparing the instances for training.........
2019-06-26 13:39:12 Downloading - Downloading input data
2019-06-26 13:39:12 Training - Downloading the training image..
Docker entrypoint called with argument(s): train
[06/26/2019 13:39:28 INFO 139702039017280] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall'


2019-06-26 13:39:42 Uploading - Uploading generated training model
2019-06-26 13:39:42 Completed - Training job completed
Billable seconds: 47
CPU times: user 562 ms, sys: 13 ms, total: 575 ms
Wall time: 3min 48s


Now we can compare the amount of time billed and the accuracy compared to our baseline model.